# ESS refactoring
Builds on top of xarray-einstats.

In [1]:
import xarray as xr
import arviz as az
import numpy as np
from scipy.fftpack import next_fast_len

from utils import autocov, autocorr, _ess_mean

In [2]:
ds = az.load_arviz_data("centered_eight").posterior
ds

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       (chain, draw) float64 ...
    theta    (chain, draw, school) float64 ...
    tau      (chain, draw) float64 ...
Attributes:
    created_at:                 2019-06-21T17:36:34.398087
    inference_library:          pymc3
    inference_library_version:  3.7

In [3]:
xe_cov = autocov(ds.theta)
az_cov = az.autocov(ds.theta.values, axis=1)
np.allclose(xe_cov.transpose(*ds.theta.dims), az_cov)

True

In [4]:
xe_corr = autocorr(ds.theta)
az_corr = az.autocorr(ds.theta.values, axis=1)
np.allclose(xe_corr.transpose(*ds.theta.dims), az_corr)

True

In [5]:
ds = az.load_arviz_data("centered_eight").posterior
#ds.theta.loc[dict(school="Choate")] = 0
_ess_mean(ds.theta.chunk(chunks=dict(school=2)), dask="allowed").compute()

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    theta    (school) float64 394.2 567.1 503.8 613.3 393.0 554.8 527.1 573.6

In [6]:
az.ess(ds, method="mean")

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       float64 246.4
    theta    (school) float64 394.2 567.1 503.8 613.3 393.0 554.8 527.1 573.6
    tau      float64 142.3